In [3]:
import json
import pandas as pd
import re

# Caminho do notebook
arquivo_notebook = "Automacao_Email_Mkt.ipynb"

# Abrir e carregar JSON
with open(arquivo_notebook, "r", encoding="utf-8") as f:
    notebook = json.load(f)

# Lista para armazenar linhas do output
linhas = []

# Percorrer células do tipo 'code'
for cell in notebook["cells"]:
    if cell["cell_type"] == "code":
        for output in cell.get("outputs", []):
            if output["output_type"] == "stream":
                linhas.extend(output["text"])

# Agora 'linhas' tem todo o stdout/stderr do notebook
# Podemos aplicar a mesma lógica de regex que te passei antes

sucessos = []
erros = []

regex_sucesso = re.compile(r"✅ E-mail enviado para (.+) \((.+)\) - (.+)")
regex_erro = re.compile(r"❌ Erro ao enviar para (.+) \((.*)\): (.+)")

for linha in linhas:
    linha = linha.strip()
    match_sucesso = regex_sucesso.search(linha)
    match_erro = regex_erro.search(linha)
    
    if match_sucesso:
        sucessos.append({
            "Nome": match_sucesso.group(1),
            "Email": match_sucesso.group(2),
            "Empresa": match_sucesso.group(3),
        })
    elif match_erro:
        erros.append({
            "Nome": match_erro.group(1),
            "Email": match_erro.group(2),
            "Erro": match_erro.group(3),
        })

# Criar DataFrames
df_sucessos = pd.DataFrame(sucessos)
df_erros = pd.DataFrame(erros)

# Salvar em Excel com duas abas
with pd.ExcelWriter("Relatorio_Ultimo_Disparo.xlsx") as writer:
    df_sucessos.to_excel(writer, sheet_name="Sucesso", index=False)
    df_erros.to_excel(writer, sheet_name="Erros", index=False)

print("✅ Relatório criado: 'Relatorio_Ultimo_Disparo.xlsx' com abas Sucesso e Erros")


✅ Relatório criado: 'Relatorio_Ultimo_Disparo.xlsx' com abas Sucesso e Erros
